<center><h1>Hong_Seungil_HW7</h1></center>

Name: Seungil Hong
<br>
Github Username: zonna19
<br>
USC ID: 2375832093 

## 1. Multi-class and Multi-Label Classification Using Support Vector Machines

Import packages

In [12]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

### (a) Download the Anuran Calls (MFCCs) Data Set

In [4]:
# Data read
df_raw = pd.read_csv('..\data\Frogs_MFCCs.csv')

In [5]:
df_raw

,MFCCs_ 1,MFCCs_ 2,MFCCs_ 3,MFCCs_ 4,MFCCs_ 5,MFCCs_ 6,MFCCs_ 7,MFCCs_ 8,MFCCs_ 9,MFCCs_10,...,MFCCs_17,MFCCs_18,MFCCs_19,MFCCs_20,MFCCs_21,MFCCs_22,Family,Genus,Species,RecordID
0,1.0,0.152936,-0.105586,0.200722,0.317201,0.260764,0.100945,-0.150063,-0.171128,0.124676,...,-0.108351,-0.077623,-0.009568,0.057684,0.118680,0.014038,Leptodactylidae,Adenomera,AdenomeraAndre,1
1,1.0,0.171534,-0.098975,0.268425,0.338672,0.268353,0.060835,-0.222475,-0.207693,0.170883,...,-0.090974,-0.056510,-0.035303,0.020140,0.082263,0.029056,Leptodactylidae,Adenomera,AdenomeraAndre,1
2,1.0,0.152317,-0.082973,0.287128,0.276014,0.189867,0.008714,-0.242234,-0.219153,0.232538,...,-0.050691,-0.023590,-0.066722,-0.025083,0.099108,0.077162,Leptodactylidae,Adenomera,AdenomeraAndre,1
3,1.0,0.224392,0.118985,0.329432,0.372088,0.361005,0.015501,-0.194347,-0.098181,0.270375,...,-0.136009,-0.177037,-0.130498,-0.054766,-0.018691,0.023954,Leptodactylidae,Adenomera,AdenomeraAndre,1
4,1.0,0.087817,-0.068345,0.306967,0.330923,0.249144,0.006884,-0.265423,-0.172700,0.266434,...,-0.048885,-0.053074,-0.088550,-0.031346,0.108610,0.079244,Leptodactylidae,Adenomera,AdenomeraAndre,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7190,1.0,-0.554504,-0.337717,0.035533,0.034511,0.443451,0.093889,-0.100753,0.037087,0.081075,...,0.069430,0.071001,0.021591,0.052449,-0.021860,-0.079860,Hylidae,Scinax,ScinaxRuber,60
7191,1.0,-0.517273,-0.370574,0.030673,0.068097,0.402890,0.096628,-0.116460,0.063727,0.089034,...,0.061127,0.068978,0.017745,0.046461,-0.015418,-0.101892,Hylidae,Scinax,ScinaxRuber,60
7192,1.0,-0.582557,-0.343237,0.029468,0.064179,0.385596,0.114905,-0.103317,0.070370,0.081317,...,0.082474,0.077771,-0.009688,0.027834,-0.000531,-0.080425,Hylidae,Scinax,ScinaxRuber,60
7193,1.0,-0.519497,-0.307553,-0.004922,0.072865,0.377131,0.086866,-0.115799,0.056979,0.089316,...,0.051796,0.069073,0.017963,0.041803,-0.027911,-0.096895,Hylidae,Scinax,ScinaxRuber,60


In [7]:
# Identify features with missing values
missing_data = df_raw.isnull().sum()

# print columns with missing values
print("Missing Data columns\n", missing_data[missing_data > 0],"\n==========================")  


Missing Data columns
 Series([], dtype: int64) 


In [9]:
# Split data into training and test sets (30%)
raw_tr, raw_te = train_test_split(df_raw, test_size=0.30, random_state=42)

### (b) Train a classifier for each label

#### (i) Research

> Exact Match and Hamming Loss are two methods used for evaluating multi-label classification models:

> * Exact Match (also known as Subset Accuracy): This is the most strict metric, as it requires the predicted set of labels to exactly match the true set of labels for each sample. It's a harsh metric because it doesn't give any partial credit for being partially correct.

> * Hamming Loss: This calculates the average Hamming loss or Hamming distance between two sets of samples. The Hamming loss is the fraction of labels that are incorrectly predicted, so it's a measure of how many label predictions were incorrect out of all label predictions made. It is more forgiving than the Exact Match because it doesn't require all labels to be correct for the sample to be considered correct. Instead, it penalizes only the individual incorrect labels​​​​.

> Reference https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html

In [59]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import make_scorer, accuracy_score, classification_report, hamming_loss
from sklearn.preprocessing import MultiLabelBinarizer



#### (ii) Train a SVM for each of the labels

In [43]:
# Split Attributes and Classifications
# For train
X_tr_raw = raw_tr.iloc[:, :-4]

# For test
X_te_raw = raw_te.iloc[:, :-4]

* Non-standardized dataset

In [53]:
# Define the parameter grid for 'C' and 'gamma'
param_grid = {
    'C': [1, 10, 100, 1000 ],  # Example values
    'gamma': [1, 2, 3, 4]  # Example values
}

# Initialize the scorer and collect predictions
scorer = make_scorer(accuracy_score)

num_labels = 3  # You have three labels: Family, Genus, Species
num_samples = raw_te.shape[0]  # Number of samples in your test set

# Initialize a matrix to hold all predictions
all_predictions = np.empty((num_samples, num_labels), dtype=object)


In [54]:
# Initialize and train the SVMs with cross-validation for each label
for i, label in enumerate(['Family', 'Genus', 'Species']):
    y_train = raw_tr[label]
    svm = SVC(kernel='rbf')
    clf = GridSearchCV(svm, param_grid, cv=10, scoring=scorer)
    clf.fit(X_tr_raw, y_train)
    best_C = clf.best_params_['C']
    best_gamma = clf.best_params_['gamma']
    print(f"Best params for {label}: C={best_C}, gamma={best_gamma}")

    svm_best = SVC(kernel='rbf', C=best_C, gamma=best_gamma)
    svm_best.fit(X_tr_raw, y_train)
    y_te_pred = svm_best.predict(X_te_raw)
    all_predictions[:, i] = y_te_pred  # Fill the column with predictions for the current label
    te_accuracy = accuracy_score(raw_te[label], y_te_pred)
    
    print(f"Test Accuracy for {label}: {te_accuracy:.4f}")
    print(classification_report(raw_te[label], y_te_pred))

Best params for Family: C=10, gamma=2
Test Accuracy for Family: 0.9940
                 precision    recall  f1-score   support

      Bufonidae       1.00      0.89      0.94        27
  Dendrobatidae       0.99      1.00      1.00       162
        Hylidae       0.99      1.00      0.99       623
Leptodactylidae       1.00      0.99      1.00      1347

       accuracy                           0.99      2159
      macro avg       0.99      0.97      0.98      2159
   weighted avg       0.99      0.99      0.99      2159

Best params for Genus: C=10, gamma=3
Test Accuracy for Genus: 0.9898
               precision    recall  f1-score   support

    Adenomera       0.99      1.00      1.00      1251
     Ameerega       0.99      1.00      1.00       162
Dendropsophus       1.00      0.96      0.98        84
    Hypsiboas       0.98      1.00      0.99       468
Leptodactylus       1.00      0.97      0.98        96
Osteocephalus       0.93      0.82      0.87        34
     Rhinella  

In [62]:
# Instantiate the binarizer
mlb = MultiLabelBinarizer()

# Binarize the true labels
true_labels_binarized = mlb.fit_transform(raw_te[['Family', 'Genus', 'Species']].values)

# Binarize the predictions
predictions_binarized = mlb.transform(all_predictions)

# Calculate Exact Match (Subset Accuracy) across all labels
exact_matches = np.all(true_labels_binarized == predictions_binarized, axis=1)
exact_match_score = np.mean(exact_matches)
print(f'Exact Match : {exact_match_score:.4f}')

# Calculate Hamming Loss across all labels using the binarized data
hamm_loss = hamming_loss(true_labels_binarized, predictions_binarized)
print(f'Hamming Loss: {hamm_loss:.4f}')

Exact Match : 0.9861
Hamming Loss: 0.0025


> * Exact Match (Subset Accuracy): 0.9861
>> This score implies that in 98.61% of instances, the model's predictions were entirely correct across all labels. In other words, for nearly all the samples, every predicted label matched the true label. A sample's predicted label set to be exactly the same as the true set to be considered correct. 
> * Hamming Loss: 0.0025
>> The Hamming Loss is the fraction of labels that are incorrectly predicted, which means it's the average number of label errors per sample. A lower score is better. Specifically, it means that, on average, only 0.25% of the labels were incorrectly predicted.

* Standardize the dataset

In [63]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_tr_scale = scaler.fit_transform(X_tr_raw)
X_te_scale = scaler.fit_transform(X_te_raw)

In [66]:
# Do the same with standardized datasets
param_grid = {
    'C': [1, 10, 100, 1000 ],  # Example values
    'gamma': [0.001, 0.01, 0.1, 1, 2, 3]  # Example values
}

num_labels = 3  # You have three labels: Family, Genus, Species
num_samples = raw_te.shape[0]  # Number of samples in your test set

# Initialize a matrix to hold all predictions
all_predictions = np.empty((num_samples, num_labels), dtype=object)

# Initialize the scorer
scorer = make_scorer(accuracy_score)

# Initialize and train the SVMs with cross-validation for each label
for i, label in enumerate(['Family', 'Genus', 'Species']):
    y_train = raw_tr[label]
    svm = SVC(kernel='rbf')
    clf = GridSearchCV(svm, param_grid, cv=10, scoring=scorer)
    clf.fit(X_tr_scale, y_train)
    best_C = clf.best_params_['C']
    best_gamma = clf.best_params_['gamma']
    print(f"Best params for {label}: C={best_C}, gamma={best_gamma}")

    svm_best = SVC(kernel='rbf', C=best_C, gamma=best_gamma)
    svm_best.fit(X_tr_scale, y_train)
    y_te_pred = svm_best.predict(X_te_scale)
    all_predictions[:, i] = y_te_pred  # Fill the column with predictions for the current label
    te_accuracy = accuracy_score(raw_te[label], y_te_pred)
    
    print(f"Test Accuracy for {label}: {te_accuracy:.4f}")
    print(classification_report(raw_te[label], y_te_pred))

Best params for Family: C=10, gamma=0.1
Test Accuracy for Family: 0.9935
                 precision    recall  f1-score   support

      Bufonidae       1.00      0.89      0.94        27
  Dendrobatidae       1.00      1.00      1.00       162
        Hylidae       0.98      1.00      0.99       623
Leptodactylidae       1.00      0.99      1.00      1347

       accuracy                           0.99      2159
      macro avg       0.99      0.97      0.98      2159
   weighted avg       0.99      0.99      0.99      2159

Best params for Genus: C=10, gamma=0.1
Test Accuracy for Genus: 0.9884
               precision    recall  f1-score   support

    Adenomera       1.00      1.00      1.00      1251
     Ameerega       1.00      1.00      1.00       162
Dendropsophus       1.00      0.96      0.98        84
    Hypsiboas       0.96      1.00      0.98       468
Leptodactylus       1.00      0.96      0.98        96
Osteocephalus       0.90      0.82      0.86        34
     Rhinel

In [67]:
# Instantiate the binarizer
mlb = MultiLabelBinarizer()

# Binarize the true labels
true_labels_binarized = mlb.fit_transform(raw_te[['Family', 'Genus', 'Species']].values)

# Binarize the predictions
predictions_binarized = mlb.transform(all_predictions)

# Calculate Exact Match (Subset Accuracy) across all labels
exact_matches = np.all(true_labels_binarized == predictions_binarized, axis=1)
exact_match_score = np.mean(exact_matches)
print(f'Exact Match : {exact_match_score:.4f}')

# Calculate Hamming Loss across all labels using the binarized data
hamm_loss = hamming_loss(true_labels_binarized, predictions_binarized)
print(f'Hamming Loss: {hamm_loss:.4f}')

Exact Match : 0.9824
Hamming Loss: 0.0027


#### (iii) Repeat 1(b)ii with L1-penalized SVMs

#### (iv) Repeat 1(b)iii by using SMOTE or any other method for imbalance

## 2. K-Means Clustering on a Multi-Class and Multi-Label Data Set

### (a) Use k-means clustering

### (b) Determine which family is the majority

### (c) Calculate the average Hamming distance, Hamming score, and Hamming loss

## 3. ISLR 12.6.2